# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())
            self.model.add(dropout(rate=0.5))

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.1360, train_loss: 2.36265612, train_accuracy: 0.2500, test_Accuracy: 0.1572
Epoch: [ 0] [    1/  468] time: 1.2536, train_loss: 2.33622384, train_accuracy: 0.2344, test_Accuracy: 0.2088
Epoch: [ 0] [    2/  468] time: 1.3634, train_loss: 2.31260991, train_accuracy: 0.3906, test_Accuracy: 0.3332
Epoch: [ 0] [    3/  468] time: 1.4731, train_loss: 2.30431223, train_accuracy: 0.3047, test_Accuracy: 0.3576
Epoch: [ 0] [    4/  468] time: 1.5768, train_loss: 2.22258639, train_accuracy: 0.4062, test_Accuracy: 0.3920
Epoch: [ 0] [    5/  468] time: 1.6745, train_loss: 2.23002505, train_accuracy: 0.4688, test_Accuracy: 0.4297
Epoch: [ 0] [    6/  468] time: 1.7772, train_loss: 2.17408633, train_accuracy: 0.4531, test_Accuracy: 0.4577
Epoch: [ 0] [    7/  468] time: 1.8999, train_loss: 2.15135169, train_accuracy: 0.4688, test_Accuracy: 0.5062
Epoch: [ 0] [    8/  468] time: 2.0026,

Epoch: [ 0] [   74/  468] time: 8.9830, train_loss: 0.43151414, train_accuracy: 0.9375, test_Accuracy: 0.8882
Epoch: [ 0] [   75/  468] time: 9.0887, train_loss: 0.82927388, train_accuracy: 0.8594, test_Accuracy: 0.8838
Epoch: [ 0] [   76/  468] time: 9.1954, train_loss: 0.91739690, train_accuracy: 0.8047, test_Accuracy: 0.8850
Epoch: [ 0] [   77/  468] time: 9.2931, train_loss: 0.58663726, train_accuracy: 0.9141, test_Accuracy: 0.8911
Epoch: [ 0] [   78/  468] time: 9.3989, train_loss: 0.51586044, train_accuracy: 0.8906, test_Accuracy: 0.8951
Epoch: [ 0] [   79/  468] time: 9.5116, train_loss: 0.89493966, train_accuracy: 0.8516, test_Accuracy: 0.8968
Epoch: [ 0] [   80/  468] time: 9.6223, train_loss: 0.60667467, train_accuracy: 0.8984, test_Accuracy: 0.8911
Epoch: [ 0] [   81/  468] time: 9.7260, train_loss: 0.58950299, train_accuracy: 0.8984, test_Accuracy: 0.8890
Epoch: [ 0] [   82/  468] time: 9.8297, train_loss: 0.62955153, train_accuracy: 0.8828, test_Accuracy: 0.8858
Epoch: [ 0

Epoch: [ 0] [  149/  468] time: 17.0165, train_loss: 0.19730026, train_accuracy: 0.9688, test_Accuracy: 0.9310
Epoch: [ 0] [  150/  468] time: 17.1332, train_loss: 0.48746109, train_accuracy: 0.9219, test_Accuracy: 0.9292
Epoch: [ 0] [  151/  468] time: 17.2409, train_loss: 0.45484269, train_accuracy: 0.8984, test_Accuracy: 0.9277
Epoch: [ 0] [  152/  468] time: 17.3446, train_loss: 0.48753989, train_accuracy: 0.9219, test_Accuracy: 0.9282
Epoch: [ 0] [  153/  468] time: 17.4533, train_loss: 0.29132497, train_accuracy: 0.9375, test_Accuracy: 0.9278
Epoch: [ 0] [  154/  468] time: 17.5620, train_loss: 0.31591210, train_accuracy: 0.9453, test_Accuracy: 0.9271
Epoch: [ 0] [  155/  468] time: 17.6667, train_loss: 0.39297146, train_accuracy: 0.9297, test_Accuracy: 0.9277
Epoch: [ 0] [  156/  468] time: 17.7685, train_loss: 0.39329702, train_accuracy: 0.9062, test_Accuracy: 0.9274
Epoch: [ 0] [  157/  468] time: 17.8772, train_loss: 0.25655320, train_accuracy: 0.9609, test_Accuracy: 0.9254
E

Epoch: [ 0] [  223/  468] time: 24.9712, train_loss: 0.23380205, train_accuracy: 0.9531, test_Accuracy: 0.9389
Epoch: [ 0] [  224/  468] time: 25.0769, train_loss: 0.20867100, train_accuracy: 0.9766, test_Accuracy: 0.9395
Epoch: [ 0] [  225/  468] time: 25.1816, train_loss: 0.30082172, train_accuracy: 0.9609, test_Accuracy: 0.9395
Epoch: [ 0] [  226/  468] time: 25.2864, train_loss: 0.27285996, train_accuracy: 0.9531, test_Accuracy: 0.9381
Epoch: [ 0] [  227/  468] time: 25.3951, train_loss: 0.58272451, train_accuracy: 0.8984, test_Accuracy: 0.9377
Epoch: [ 0] [  228/  468] time: 25.4998, train_loss: 0.47645795, train_accuracy: 0.9062, test_Accuracy: 0.9370
Epoch: [ 0] [  229/  468] time: 25.6035, train_loss: 0.48423278, train_accuracy: 0.8984, test_Accuracy: 0.9372
Epoch: [ 0] [  230/  468] time: 25.7132, train_loss: 0.39864868, train_accuracy: 0.9141, test_Accuracy: 0.9379
Epoch: [ 0] [  231/  468] time: 25.8189, train_loss: 0.34766334, train_accuracy: 0.9297, test_Accuracy: 0.9392
E

Epoch: [ 0] [  297/  468] time: 32.9020, train_loss: 0.33010226, train_accuracy: 0.9141, test_Accuracy: 0.9477
Epoch: [ 0] [  298/  468] time: 33.0057, train_loss: 0.46310985, train_accuracy: 0.9453, test_Accuracy: 0.9460
Epoch: [ 0] [  299/  468] time: 33.1094, train_loss: 0.30524984, train_accuracy: 0.9297, test_Accuracy: 0.9465
Epoch: [ 0] [  300/  468] time: 33.2162, train_loss: 0.47459191, train_accuracy: 0.9062, test_Accuracy: 0.9463
Epoch: [ 0] [  301/  468] time: 33.3209, train_loss: 0.27233392, train_accuracy: 0.9375, test_Accuracy: 0.9466
Epoch: [ 0] [  302/  468] time: 33.4386, train_loss: 0.19476581, train_accuracy: 0.9609, test_Accuracy: 0.9467
Epoch: [ 0] [  303/  468] time: 33.5433, train_loss: 0.34703478, train_accuracy: 0.9297, test_Accuracy: 0.9471
Epoch: [ 0] [  304/  468] time: 33.6530, train_loss: 0.31242323, train_accuracy: 0.9375, test_Accuracy: 0.9477
Epoch: [ 0] [  305/  468] time: 33.7697, train_loss: 0.26316145, train_accuracy: 0.9219, test_Accuracy: 0.9497
E

Epoch: [ 0] [  371/  468] time: 40.8108, train_loss: 0.21954784, train_accuracy: 0.9688, test_Accuracy: 0.9543
Epoch: [ 0] [  372/  468] time: 40.9176, train_loss: 0.15280977, train_accuracy: 0.9609, test_Accuracy: 0.9547
Epoch: [ 0] [  373/  468] time: 41.0273, train_loss: 0.24171659, train_accuracy: 0.9531, test_Accuracy: 0.9537
Epoch: [ 0] [  374/  468] time: 41.1300, train_loss: 0.23733643, train_accuracy: 0.9688, test_Accuracy: 0.9528
Epoch: [ 0] [  375/  468] time: 41.2327, train_loss: 0.19830006, train_accuracy: 0.9453, test_Accuracy: 0.9522
Epoch: [ 0] [  376/  468] time: 41.3374, train_loss: 0.18881527, train_accuracy: 0.9766, test_Accuracy: 0.9513
Epoch: [ 0] [  377/  468] time: 41.4461, train_loss: 0.26002574, train_accuracy: 0.9609, test_Accuracy: 0.9508
Epoch: [ 0] [  378/  468] time: 41.5499, train_loss: 0.25464520, train_accuracy: 0.9609, test_Accuracy: 0.9521
Epoch: [ 0] [  379/  468] time: 41.6456, train_loss: 0.35757867, train_accuracy: 0.9375, test_Accuracy: 0.9518
E

Epoch: [ 0] [  445/  468] time: 48.6080, train_loss: 0.19750543, train_accuracy: 0.9219, test_Accuracy: 0.9530
Epoch: [ 0] [  446/  468] time: 48.7127, train_loss: 0.33304739, train_accuracy: 0.9375, test_Accuracy: 0.9535
Epoch: [ 0] [  447/  468] time: 48.8194, train_loss: 0.16322465, train_accuracy: 0.9766, test_Accuracy: 0.9545
Epoch: [ 0] [  448/  468] time: 48.9212, train_loss: 0.26533374, train_accuracy: 0.9609, test_Accuracy: 0.9561
Epoch: [ 0] [  449/  468] time: 49.0259, train_loss: 0.40472245, train_accuracy: 0.9531, test_Accuracy: 0.9540
Epoch: [ 0] [  450/  468] time: 49.1386, train_loss: 0.21256977, train_accuracy: 0.9766, test_Accuracy: 0.9543
Epoch: [ 0] [  451/  468] time: 49.2403, train_loss: 0.18151399, train_accuracy: 0.9844, test_Accuracy: 0.9543
Epoch: [ 0] [  452/  468] time: 49.3440, train_loss: 0.17057095, train_accuracy: 0.9375, test_Accuracy: 0.9544
Epoch: [ 0] [  453/  468] time: 49.4477, train_loss: 0.30449033, train_accuracy: 0.9453, test_Accuracy: 0.9547
E

# Test accuracy : 95.91 %

# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm() :
    return tf.keras.layers.BatchNormalization()

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.1689, train_loss: 0.35015073, train_accuracy: 0.4219, test_Accuracy: 0.3032
Epoch: [ 0] [    1/  468] time: 1.2955, train_loss: 0.37097424, train_accuracy: 0.5156, test_Accuracy: 0.4046
Epoch: [ 0] [    2/  468] time: 1.4202, train_loss: 0.29386908, train_accuracy: 0.5234, test_Accuracy: 0.4384
Epoch: [ 0] [    3/  468] time: 1.5548, train_loss: 0.37836087, train_accuracy: 0.4844, test_Accuracy: 0.4641
Epoch: [ 0] [    4/  468] time: 1.6845, train_loss: 0.28141460, train_accuracy: 0.5156, test_Accuracy: 0.4928
Epoch: [ 0] [    5/  468] time: 1.8082, train_loss: 0.30122110, train_accuracy: 0.5781, test_Accuracy: 0.5240
Epoch: [ 0] [    6/  468] time: 1.9408, train_loss: 0.42284572, train_accuracy: 0.5234, test_Accuracy: 0.5506
Epoch: [ 0] [    7/  468] time: 2.0705, train_loss: 0.28941354, train_accuracy: 0.5547, test_Accuracy: 0.5829
Epoch: [ 0] [    8/  468] time: 2.1951,

Epoch: [ 0] [   74/  468] time: 10.5967, train_loss: 0.10039282, train_accuracy: 0.9375, test_Accuracy: 0.9231
Epoch: [ 0] [   75/  468] time: 10.7203, train_loss: 0.11034554, train_accuracy: 0.9453, test_Accuracy: 0.9240
Epoch: [ 0] [   76/  468] time: 10.8520, train_loss: 0.17930251, train_accuracy: 0.9062, test_Accuracy: 0.9267
Epoch: [ 0] [   77/  468] time: 10.9776, train_loss: 0.17978302, train_accuracy: 0.9375, test_Accuracy: 0.9289
Epoch: [ 0] [   78/  468] time: 11.1043, train_loss: 0.14655173, train_accuracy: 0.9531, test_Accuracy: 0.9321
Epoch: [ 0] [   79/  468] time: 11.2270, train_loss: 0.12640114, train_accuracy: 0.9375, test_Accuracy: 0.9352
Epoch: [ 0] [   80/  468] time: 11.3576, train_loss: 0.09021215, train_accuracy: 0.9609, test_Accuracy: 0.9370
Epoch: [ 0] [   81/  468] time: 11.4813, train_loss: 0.15853114, train_accuracy: 0.9219, test_Accuracy: 0.9362
Epoch: [ 0] [   82/  468] time: 11.6070, train_loss: 0.12031908, train_accuracy: 0.9609, test_Accuracy: 0.9355
E

Epoch: [ 0] [  148/  468] time: 20.0733, train_loss: 0.07749590, train_accuracy: 0.9766, test_Accuracy: 0.9559
Epoch: [ 0] [  149/  468] time: 20.1970, train_loss: 0.10683800, train_accuracy: 0.9688, test_Accuracy: 0.9555
Epoch: [ 0] [  150/  468] time: 20.3296, train_loss: 0.15772223, train_accuracy: 0.9453, test_Accuracy: 0.9550
Epoch: [ 0] [  151/  468] time: 20.4533, train_loss: 0.08395267, train_accuracy: 0.9766, test_Accuracy: 0.9552
Epoch: [ 0] [  152/  468] time: 20.5819, train_loss: 0.08575320, train_accuracy: 0.9375, test_Accuracy: 0.9559
Epoch: [ 0] [  153/  468] time: 20.7176, train_loss: 0.07256294, train_accuracy: 0.9688, test_Accuracy: 0.9556
Epoch: [ 0] [  154/  468] time: 20.8452, train_loss: 0.18948507, train_accuracy: 0.9297, test_Accuracy: 0.9554
Epoch: [ 0] [  155/  468] time: 20.9699, train_loss: 0.11677647, train_accuracy: 0.9453, test_Accuracy: 0.9559
Epoch: [ 0] [  156/  468] time: 21.1006, train_loss: 0.12934691, train_accuracy: 0.9453, test_Accuracy: 0.9570
E

Epoch: [ 0] [  222/  468] time: 29.5051, train_loss: 0.08141253, train_accuracy: 0.9609, test_Accuracy: 0.9608
Epoch: [ 0] [  223/  468] time: 29.6307, train_loss: 0.10475951, train_accuracy: 0.9531, test_Accuracy: 0.9607
Epoch: [ 0] [  224/  468] time: 29.7644, train_loss: 0.09060709, train_accuracy: 0.9766, test_Accuracy: 0.9607
Epoch: [ 0] [  225/  468] time: 29.8881, train_loss: 0.11126640, train_accuracy: 0.9531, test_Accuracy: 0.9594
Epoch: [ 0] [  226/  468] time: 30.0137, train_loss: 0.09707939, train_accuracy: 0.9688, test_Accuracy: 0.9591
Epoch: [ 0] [  227/  468] time: 30.1444, train_loss: 0.10313089, train_accuracy: 0.9766, test_Accuracy: 0.9590
Epoch: [ 0] [  228/  468] time: 30.2700, train_loss: 0.05771070, train_accuracy: 0.9844, test_Accuracy: 0.9600
Epoch: [ 0] [  229/  468] time: 30.3937, train_loss: 0.06698586, train_accuracy: 0.9766, test_Accuracy: 0.9615
Epoch: [ 0] [  230/  468] time: 30.5224, train_loss: 0.11574902, train_accuracy: 0.9531, test_Accuracy: 0.9620
E

Epoch: [ 0] [  296/  468] time: 38.8870, train_loss: 0.04859398, train_accuracy: 0.9844, test_Accuracy: 0.9592
Epoch: [ 0] [  297/  468] time: 39.0117, train_loss: 0.02990494, train_accuracy: 1.0000, test_Accuracy: 0.9587
Epoch: [ 0] [  298/  468] time: 39.1383, train_loss: 0.09809260, train_accuracy: 0.9766, test_Accuracy: 0.9590
Epoch: [ 0] [  299/  468] time: 39.2630, train_loss: 0.06100036, train_accuracy: 0.9688, test_Accuracy: 0.9585
Epoch: [ 0] [  300/  468] time: 39.3926, train_loss: 0.06691900, train_accuracy: 0.9844, test_Accuracy: 0.9576
Epoch: [ 0] [  301/  468] time: 39.5153, train_loss: 0.04413445, train_accuracy: 0.9609, test_Accuracy: 0.9575
Epoch: [ 0] [  302/  468] time: 39.6410, train_loss: 0.12962659, train_accuracy: 0.9688, test_Accuracy: 0.9590
Epoch: [ 0] [  303/  468] time: 39.7656, train_loss: 0.09929971, train_accuracy: 0.9766, test_Accuracy: 0.9608
Epoch: [ 0] [  304/  468] time: 39.8953, train_loss: 0.11513779, train_accuracy: 0.9844, test_Accuracy: 0.9619
E

Epoch: [ 0] [  370/  468] time: 48.3128, train_loss: 0.10199833, train_accuracy: 0.9766, test_Accuracy: 0.9686
Epoch: [ 0] [  371/  468] time: 48.4364, train_loss: 0.15238087, train_accuracy: 0.9531, test_Accuracy: 0.9697
Epoch: [ 0] [  372/  468] time: 48.5581, train_loss: 0.10613895, train_accuracy: 0.9766, test_Accuracy: 0.9707
Epoch: [ 0] [  373/  468] time: 48.6968, train_loss: 0.06351810, train_accuracy: 0.9609, test_Accuracy: 0.9725
Epoch: [ 0] [  374/  468] time: 48.8204, train_loss: 0.03646889, train_accuracy: 0.9766, test_Accuracy: 0.9728
Epoch: [ 0] [  375/  468] time: 48.9491, train_loss: 0.07110221, train_accuracy: 0.9922, test_Accuracy: 0.9724
Epoch: [ 0] [  376/  468] time: 49.0787, train_loss: 0.11963235, train_accuracy: 0.9609, test_Accuracy: 0.9714
Epoch: [ 0] [  377/  468] time: 49.2114, train_loss: 0.05185366, train_accuracy: 0.9766, test_Accuracy: 0.9714
Epoch: [ 0] [  378/  468] time: 49.3380, train_loss: 0.03372043, train_accuracy: 0.9922, test_Accuracy: 0.9723
E

Epoch: [ 0] [  444/  468] time: 57.6937, train_loss: 0.05690372, train_accuracy: 0.9844, test_Accuracy: 0.9683
Epoch: [ 0] [  445/  468] time: 57.8174, train_loss: 0.11636702, train_accuracy: 0.9609, test_Accuracy: 0.9685
Epoch: [ 0] [  446/  468] time: 57.9470, train_loss: 0.07634787, train_accuracy: 0.9688, test_Accuracy: 0.9675
Epoch: [ 0] [  447/  468] time: 58.0747, train_loss: 0.04071792, train_accuracy: 0.9922, test_Accuracy: 0.9678
Epoch: [ 0] [  448/  468] time: 58.2013, train_loss: 0.04820122, train_accuracy: 0.9922, test_Accuracy: 0.9686
Epoch: [ 0] [  449/  468] time: 58.3240, train_loss: 0.07842625, train_accuracy: 0.9766, test_Accuracy: 0.9685
Epoch: [ 0] [  450/  468] time: 58.4527, train_loss: 0.05513705, train_accuracy: 0.9844, test_Accuracy: 0.9692
Epoch: [ 0] [  451/  468] time: 58.5753, train_loss: 0.06282906, train_accuracy: 0.9766, test_Accuracy: 0.9692
Epoch: [ 0] [  452/  468] time: 58.6990, train_loss: 0.09079029, train_accuracy: 0.9766, test_Accuracy: 0.9702
E

# Test accuracy : 97.51 %